# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
import numpy as np
import pandas as pd
import zipfile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
with zipfile.ZipFile("Orders.zip") as zip_ref:
    zip_ref.extractall()

orders = pd.read_csv("Orders.csv")
orders.head(10)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
5,5,536365,22752,2010,12,3,8,set 7 babushka nesting boxes,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,15.30
6,6,536365,21730,2010,12,3,8,glass star frosted t-light holder,6,2010-12-01 08:26:00,4.25,17850,United Kingdom,25.50
7,7,536366,22633,2010,12,3,8,hand warmer union jack,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,11.10
8,8,536366,22632,2010,12,3,8,hand warmer red polka dot,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,11.10
9,9,536367,84879,2010,12,3,8,assorted colour bird ornament,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,54.08


"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# Sub-problem 1 Steps: group them by CustomerID and then sum the orders to find the amount_spend per customer

orders.sample(2)

,index,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
13476,13476,20363,538040,21166,2010,12,4,13,cook with wine metal sign,3,2010-12-09 13:08:00,1.95,16520,United Kingdom,5.85
280655,280655,387909,570405,22233,2011,10,1,12,jigsaw rabbit and birdhouse,12,2011-10-10 12:51:00,0.59,13429,United Kingdom,7.08


In [11]:
customer_spendings = orders.groupby('CustomerID')['amount_spent'].agg('sum').reset_index()
customer_spendings.sample(5)

,CustomerID,amount_spent
1817,14823,321.43
281,12691,1035.80
1882,14913,1227.43
2742,16086,1121.39
1643,14578,168.63


In [14]:
# Sub-problem 2 Steps: find the VIPs and Preferred customers taking into account the amount spent

pref_cust = customer_spendings[(customer_spendings['amount_spent'] > customer_spendings['amount_spent'].quantile(0.75)) &
                        (customer_spendings['amount_spent'] < customer_spendings['amount_spent'].quantile(0.95))]

vip_cust = customer_spendings[(customer_spendings['amount_spent'] > customer_spendings['amount_spent'].quantile(0.95))]

In [15]:
display(pref_cust.sample(2),vip_cust.sample(2))

,CustomerID,amount_spent
3730,17451,1721.40
298,12714,4708.86


,CustomerID,amount_spent
1127,13871,6403.25
919,13576,6717.34


In [25]:
# Sub-problem 3 Steps: Add the labels VIP & Preferred to Orders

orders['customer_label'] = 0

def cust_label(client):
    if client in list(vip_cust['CustomerID']):
        return 'VIP'
    elif client in list(pref_cust['CustomerID']):
        return 'Preferred'
    else:
        return '-'
    
orders['customer_label'] = orders['CustomerID'].apply(cust_label)
orders.sort_values(by=['amount_spent'], ascending=False)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,customer_label
397451,540421,581483,23843,2011,12,5,9,"paper craft , little birdie",80995,2011-12-09 09:15:00,2.08,16446,United Kingdom,168469.60,VIP
37126,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,12346,United Kingdom,77183.60,VIP
155418,222680,556444,22502,2011,6,5,15,picnic basket wicker 60 pieces,60,2011-06-10 15:28:00,649.50,15098,United Kingdom,38970.00,VIP
118352,173382,551697,POST,2011,5,2,13,postage,1,2011-05-03 13:46:00,8142.75,16029,United Kingdom,8142.75,VIP
248706,348325,567423,23243,2011,9,2,11,set of tea coffee sugar tins pantry,1412,2011-09-20 11:05:00,5.06,17450,United Kingdom,7144.72,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334242,454463,575579,22437,2011,11,4,11,set of 9 black skull balloons,20,2011-11-10 11:49:00,0.00,13081,United Kingdom,0.00,VIP
25379,40089,539722,22423,2010,12,2,13,regency cakestand 3 tier,10,2010-12-21 13:45:00,0.00,14911,EIRE,0.00,VIP
273926,379913,569716,22778,2011,10,4,8,glass cloche small,2,2011-10-06 08:17:00,0.00,15804,United Kingdom,0.00,Preferred
353097,479546,577168,M,2011,11,5,10,manual,1,2011-11-18 10:42:00,0.00,12603,Germany,0.00,-


In [26]:
orders['customer_label'].value_counts()

Preferred    151781
-            141659
VIP          104484
Name: customer_label, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# to do later!

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# to do later!